In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint
import pandas as pd
import numpy as np
import requests

In [16]:
# Open the Bloomberg finance link for AAPL stock
page = urlopen('https://www.bloomberg.com/quote/AAPL:US').read()
soup = BeautifulSoup(page, 'html.parser')
#posts = soup.find_all('article', {'class':'newsItem__5b5cb00f'})

#<article class="newsItem__5b5cb00f  quote__23caba74"><a href="https://www.bloomberg.com/news/articles/2020-04-23/apple-aims-to-sell-macs-with-its-own-chips-starting-in-2021"><div class=" thumbnail__8ad9f836 " style="background-image:url(https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iYsQ04_.tLPE/v1/150x110.jpg)" role="img" aria-label="relates to Apple Aims to Sell Macs With Its Own Chips Starting in 2021" data-attribute-duration="--:--" wfd-id="778"></div><div class="text__ebd4e2ca withThumbnail__c3b7267c" wfd-id="774"><div class="eyebrow__b25fba9d" wfd-id="777">technology</div><div class="headline__07dbac92" wfd-id="776">Apple Aims to Sell Macs With Its Own Chips Starting in 2021</div><div class="publishedAt__4009bb4f " wfd-id="775">5 hours ago</div></div></a></article>

page

b'<!doctype html>\n<html>\n<head>\n    <title>Bloomberg - Are you a robot?</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <link rel="stylesheet" type="text/css" href="https://assets.bwbx.io/font-service/css/BWHaasGrotesk-55Roman-Web,BWHaasGrotesk-75Bold-Web,BW%20Haas%20Text%20Mono%20A-55%20Roman/font-face.css">\n    <style rel="stylesheet" type="text/css">\n        html, body, div, span, applet, object, iframe,\n        h1, h2, h3, h4, h5, h6, p, blockquote, pre,\n        a, abbr, acronym, address, big, cite, code,\n        del, dfn, em, img, ins, kbd, q, s, samp,\n        small, strike, strong, sub, sup, tt, var,\n        b, u, i, center,\n        dl, dt, dd, ol, ul, li,\n        fieldset, form, label, legend,\n        table, caption, tbody, tfoot, thead, tr, th, td,\n        article, aside, canvas, details, embed,\n        figure, figcaption, footer, header, hgroup,\n        menu, nav, output, ruby, section, summary,\n        time, mark, audio,

In [ ]:
#Then grab and save the news links into list
#urls = []
#for post in posts:
#    post_date = post.date
#    urls.append(str(post.a['href']))

# The saved links did not have 'https://finance.yahoo.com' in front of them except for the first link
# Had to remove the first link, add the pretext, and then append and print the lists to check over
#first_url = urls[0]
#urls = urls[1:]

#for url in range(len(urls)):
#    link = 'https://finance.yahoo.com'
#    urls[url] = link+urls[url]
#urls.append(first_url)

# Any new added url link does not have the pretext, so have to add it
#urls[len(urls)-1] = link+urls[len(urls)-1]

#for url in range(len(urls)):
#    print(urls[url])

In [ ]:
source = urlopen('https://www.cnbc.com/quotes/?symbol=AAPL&qsearchterm=&tab=news').read()
soup = BeautifulSoup(source, 'lxml')

parent = soup.find_all('ul')

parent

In [ ]:
latest_news = parent[21]
links = []
text = []
time = []
df = pandas.DataFrame(columns = ['links', 'text', 'timeline'])
for news in latest_news:
    if news != '\n':
        links.append(news.a['href'])
        text.append(news.span.text)
        time.append(news.find('span', class_='note'))
        
df['links'] = links
df['text'] = text
df['timeline'] = time

df

In [8]:
# Bloomberg Extractor
# weiwangchun
# using requests and beautifulsoup to grab headlines from Bloomberg


import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from lxml.html import fromstring
import sys

# grab a set proxies
# ----------------------------
def get_proxies():
    url = 'https://free-proxy-list.net/anonymous-proxy.html'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:30]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


# header settings
# ----------------------------
# ua_list:  https://udger.com/resources/ua-list
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0',
    'From': 'marcus.aurelius@rome.com' 
}


# bloomberg scraper fn
# ----------------------------
def scrape_bloomberg(subject, page_number, headers, proxies):
    full_url = 'https://www.bloomberg.com/quote/AAPL:US'
    response = requests.get(full_url, headers=headers, proxies = proxies)
    soup = BeautifulSoup(response.content, 'html.parser')
    #dates = [''.join(s.findAll(text=True))for s in soup.findAll(['time'])]
    #text = [''.join(s.findAll(text=True))for s in soup.findAll(['h1'])]
    #tmp = np.column_stack((dates, text))
    #_, idx = np.unique(tmp, axis = 0, return_index = True)
    #tmp = tmp[np.sort(idx)]
    links = [''.join(s.attrs['href']) for s in soup.findAll('a', class_="search-result-story-thumbnail__link", href = True)]
    #tmp = np.column_stack((tmp,  links))
    return links



if __name__ == '__main__':
    if len(sys.argv) != 4:
        print("Error: Please provide subject title, page_min and pages_max. \nFor Example: >Bloomberg_extractor.py bitcoin 1 99\n")
        sys.exit(1)

    subject = str(sys.argv[1])
    pages_min = int(sys.argv[2])
    pages_max = int(sys.argv[3])

    completed_page = pages_min - 1
    #full = np.array(['date', 'title', 'link'])
    full = np.array(['link'])

    while completed_page < pages_max:
        proxy_list = get_proxies()
        for proxy in proxy_list:
            pages = np.arange(completed_page + 1,pages_max+1,1)

            proxies = {
                'http': proxy,
                'https': proxy,
            }

            for page in pages:

                print('Trying Page ' + str(page) + ' using proxy ' + proxy)
                try:
                    full = np.append(full, scrape_bloomberg(subject = subject, page_number = page, headers = headers, proxies = proxies))
                    print('Page ' + str(page) + ' completed\n')
                    np.savetxt('RESULTS_TMP.csv', full, fmt='%s', delimiter=',')
                    completed_page = page
                except Exception as e:
                    print(str(e))
                    print('Failed Page ' + str(page) + ' using proxy ' + proxy)
                    break

    np.savetxt('results'+ subject + str(pages_max)+'.csv', full, fmt='%s', delimiter=',')

Error: Please provide subject title, page_min and pages_max. 
For Example: >Bloomberg_extractor.py bitcoin 1 99



SystemExit: 1

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
import time
import os
from selenium import webdriver
from bs4 import BeautifulSoup

# put the driver in the folder of this code
driver = webdriver.Chrome(os.getcwd() + '.chromedriver.exe')

WebDriverException: Message: 'NLP-Project.chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
